In [ ]:
# Cluster version: all dependencies should be installed in your virtualenv / conda env.
# No in-notebook pip installs needed here.
import os, re  # keep imports if you need them later

In [ ]:
# ============================================================
# Imports & basic config
# ============================================================
import os, json, glob, math, random, shutil, time
from pathlib import Path
from typing import List, Dict, Tuple
from collections import defaultdict

import numpy as np
import pandas as pd

import torch
from datasets import load_dataset
from huggingface_hub import snapshot_download
from scipy.spatial.distance import jensenshannon
from safetensors.torch import load_file as safe_load_file

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from peft.utils import set_peft_model_state_dict

In [ ]:
from pathlib import Path

# Absolute project root on the cluster
PROJECT_ROOT = Path("/home/wangzn/projects/aip-vshwartz/wangzn/adapter_mllm")
print("Project root:", PROJECT_ROOT)

Project root: /home/wangzn/projects/aip-vshwartz/wangzn/adapter_mllm


In [ ]:
# --------------------
# Repro & device
# --------------------
SEED = 3407
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ============================================================
# Base LM + tokenizer (also used as embedder + base baseline)
# ============================================================
BASE_MODEL_NAME = "unsloth/Meta-Llama-3.1-8B"
MAX_SEQ_LEN     = 2048
DTYPE           = None   # we'll resolve this below
LOAD_4BIT       = True

# choose a sane default dtype if not set
if DTYPE is None:
    if DEVICE == "cuda":
        # bf16 if supported, else fp16
        DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
    else:
        DTYPE = torch.float32

# Tokenizer
base_tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_NAME,
    use_fast=True,
    load_in_4bit=LOAD_4BIT,
)

if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token
base_tokenizer.padding_side = "left"
SPECIAL_IDS = set(getattr(base_tokenizer, "all_special_ids", []) or [])

# Base model (used for embeddings + baseline generation)
emb_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    dtype=DTYPE,
)
emb_model = emb_model.to(DEVICE).eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset

DATASET_NAME = "kanwal-mehreen18/urdu-code-switch-eval"
TARGET_LANG  = "ur"
MAX_EVAL     = 170 # use complete dataset afterwards

def load_eval_texts(limit: int) -> List[str]:
    ds = load_dataset(DATASET_NAME)
    print("Available splits:", ds.keys())

    split_name = list(ds.keys())[0]
    ds_split = ds[split_name]
    if len(ds_split) > limit:
        ds_split = ds_split.shuffle(seed=SEED).select(range(limit))

    texts = [t for t in ds_split["Data_cs"] if isinstance(t, str) and t.strip()]
    return texts

eval_texts = load_eval_texts(MAX_EVAL)
print(f"[Data] Loaded {len(eval_texts)} Urdu eval texts")

README.md:   0%|          | 0.00/416 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/355k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/163 [00:00<?, ? examples/s]

Available splits: dict_keys(['train'])
[Data] Loaded 163 Urdu eval texts


In [ ]:
# # ============================================================
# # Eval data (Urdu ALIF corpus)
# # ============================================================
# DATASET_NAME = "orature/ALIF_Urdu_Corpus_AUC"
# TARGET_LANG  = "ur"
# START = 0 # different across partition of job
# MAX_EVAL = 30 # set it to 5000

# def load_eval_texts(limit: int, start: int = 0) -> List[str]:
#     ds = load_dataset(DATASET_NAME, split="train")
#     end = min(start + limit, len(ds))
#     ds = ds.select(range(start, end))
#     return [t for t in ds["Data"] if isinstance(t, str) and t.strip()]

# eval_texts = load_eval_texts(MAX_EVAL, START)
# print(f"[Data] Loaded {len(eval_texts)} Urdu eval texts")

In [ ]:
# ============================================================
# Adapters on HF
# ============================================================

ADAPTERS = {
    "ur": "kanwal-mehreen18/urdu_lora_adapter",
    "ar": "anniew666/ar_lora_adapter",
}
ALL_LANGS = list(ADAPTERS.keys())  # ["ur", "ar"]

# Absolute project root on the cluster
ART_DIR  = Path.cwd() / "MLLM_adaptation/statistics"
BPE_TOPK = 5000

In [ ]:
def load_bpe_json(lang: str) -> Dict:
    p = ART_DIR / f"{lang}.bpe.top{BPE_TOPK}.json"
    with open(p, "r", encoding="utf-8") as f:
        obj = json.load(f)
    token_ids = np.array(obj["token_ids"], dtype=np.int64)
    probs     = np.array(obj["probs"], dtype=np.float64)
    probs     = probs / probs.sum() if probs.sum() > 0 else np.ones_like(probs)/len(probs)
    return {"token_ids": token_ids, "probs": probs}

def load_embed_json(lang: str) -> Dict:
    p = ART_DIR / f"{lang}.embed.json"
    with open(p, "r", encoding="utf-8") as f:
        obj = json.load(f)
    c = np.array(obj["centroid"], dtype=np.float32)
    c = c / (np.linalg.norm(c) + 1e-12)
    return {"centroid": c, "dim": int(obj["dim"])}

bpe_refs = {lg: load_bpe_json(lg) for lg in ALL_LANGS}
emb_refs = {lg: load_embed_json(lg) for lg in ALL_LANGS}
EMBED_DIM = emb_refs[TARGET_LANG]["dim"]
print(f"[Stats] Loaded BPE + centroids. EMBED_DIM={EMBED_DIM}")

[Stats] Loaded BPE + centroids. EMBED_DIM=4096


In [ ]:
# ============================================================
# Embedding + BPE similarity helpers
# ============================================================
MAX_TOKENIZE_LEN = 512

@torch.no_grad()
def embed_one(text: str) -> np.ndarray:
    enc = base_tokenizer(
        [text],
        padding=True,
        truncation=True,
        max_length=MAX_TOKENIZE_LEN,
        return_tensors="pt",
    )
    enc = {k: v.to(DEVICE) for k, v in enc.items()}
    out = emb_model(**enc, output_hidden_states=True)
    last = out.hidden_states[-1]
    mask = enc["attention_mask"].unsqueeze(-1).type_as(last)
    pooled = (last * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-6)
    pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)
    pooled = pooled.to(torch.float32)
    return pooled[0].detach().cpu().numpy()

def empirical_bpe_histogram(text: str, support_tokens: np.ndarray) -> np.ndarray:
    ids = base_tokenizer(text, add_special_tokens=False)["input_ids"]
    ids = [i for i in ids if i not in SPECIAL_IDS]
    if not ids:
        return np.ones(len(support_tokens), dtype=np.float64) / len(support_tokens)
    counts = np.zeros(len(support_tokens), dtype=np.float64)
    idx = {int(t): j for j, t in enumerate(support_tokens.tolist())}
    for tid in ids:
        j = idx.get(int(tid))
        if j is not None:
            counts[j] += 1.0
    if counts.sum() == 0:
        counts += 1e-8
    return counts / counts.sum()

def bpe_sim_jsd(p_input: np.ndarray, p_ref: np.ndarray) -> float:
    d = jensenshannon(p_input, p_ref, base=2.0)
    return 1.0 - float(d)

def cos_sim(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b) + 1e-12))

def softmax_weights(scores: List[float], temp: float = 1.0) -> List[float]:
    x = np.array(scores, dtype=np.float64) / max(1e-8, temp)
    x = x - x.max()
    w = np.exp(x)
    return (w / w.sum()).tolist()

In [ ]:
SIM_SCHEMES = {
    "bpe_only":   (1.0,  0.0),
    "emb_only":   (0.0,  1.0),
    "mix_25_75":  (0.25, 0.75),
    "mix_50_50":  (0.50, 0.50),
    "mix_75_25":  (0.75, 0.25),
}

TOP_N_PARTNERS   = 1        # we only ever want Urdu + ONE partner (Arabic)
INCLUDE_TARGET   = True
MIN_ACCEPT_SIM   = 0
SOFTMAX_TEMP     = 0.20
AGREEMENT_GUARD  = 0.10

def compute_per_lang_sims(text: str, evec: np.ndarray) -> List[Dict]:
    per_lang = []
    for lg in ALL_LANGS:   # ["ur", "ar"]
        ref_b = bpe_refs[lg]
        ref_e = emb_refs[lg]
        p_in  = empirical_bpe_histogram(text, ref_b["token_ids"])
        b_sim = bpe_sim_jsd(p_in, ref_b["probs"])
        e_sim = cos_sim(evec, ref_e["centroid"])
        per_lang.append({"lang": lg, "bpe_sim": b_sim, "emb_sim": e_sim})
    return per_lang

def select_adapters(per_lang: List[Dict], scheme: str, target_lang: str):
    alpha, beta = SIM_SCHEMES[scheme]

    scored = []
    for r in per_lang:
        b, e = r["bpe_sim"], r["emb_sim"]
        if alpha > 0 and beta > 0 and abs(b - e) > AGREEMENT_GUARD:
            s = (alpha*b + beta*e) * 0.9
        else:
            s = alpha*b + beta*e
        scored.append({**r, "scheme_score": float(max(0.0, min(1.0, s)))})

    ranked = sorted(scored, key=lambda x: x["scheme_score"], reverse=True)
    top_score = ranked[0]["scheme_score"]

    chosen = []
    if INCLUDE_TARGET and any(x["lang"] == target_lang for x in ranked):
        chosen.append(next(x for x in ranked if x["lang"] == target_lang))

    need = (1 + TOP_N_PARTNERS) if (INCLUDE_TARGET and chosen) else TOP_N_PARTNERS
    for r in ranked:
        if INCLUDE_TARGET and r["lang"] == target_lang:
            continue
        if len(chosen) >= need:
            break
        chosen.append(r)

    safety = False
    if top_score < MIN_ACCEPT_SIM:
        safety = True
        chosen = [next(x for x in ranked if x["lang"] == target_lang)] if (INCLUDE_TARGET and chosen) else [ranked[0]]

    weights = softmax_weights([c["scheme_score"] for c in chosen], temp=SOFTMAX_TEMP)
    s = sum(weights)
    weights = [w/s for w in weights]
    return chosen, weights, top_score, safety

In [ ]:
# ============================================================
# Adapter caching + merge (KeyError-safe)  -- IN-MEMORY ONLY
# ============================================================
ADAPTER_CACHE = {}

def _download_adapter(repo: str):
    if repo in ADAPTER_CACHE:
        return ADAPTER_CACHE[repo]

    local = snapshot_download(repo)
    cfg_path = Path(local) / "adapter_config.json"
    with open(cfg_path, "r") as f:
        cfg = json.load(f)

    p_bin = Path(local) / "adapter_model.bin"
    if p_bin.exists():
        sd = torch.load(str(p_bin), map_location="cpu")
    else:
        p_st = Path(local) / "adapter_model.safetensors"
        if p_st.exists():
            sd = safe_load_file(str(p_st), device="cpu")
        else:
            merged = {}
            for sf in sorted(glob.glob(str(Path(local) / "*.safetensors"))):
                merged.update(safe_load_file(sf, device="cpu"))
            sd = merged

    ADAPTER_CACHE[repo] = {"cfg": cfg, "sd": sd}
    return ADAPTER_CACHE[repo]

def _is_lora_key(k: str) -> bool:
    kk = k.lower()
    return ("lora_" in kk) or ("loraa" in kk) or ("lorab" in kk)

def soft_merge_two_sd(primary_repo: str, partner_repo: str,
                      w_primary: float, w_partner: float) -> Dict[str, torch.Tensor]:
    """
    TWO-WAY merge of LoRA weights: Urdu + Arabic.
    Returns merged state_dict (no disk writes).
    """
    prim = _download_adapter(primary_repo)
    part = _download_adapter(partner_repo)

    sds = [prim["sd"], part["sd"]]
    merged = {}

    # merge LoRA keys
    all_keys = set().union(*[sd.keys() for sd in sds])
    for k in all_keys:
        if not _is_lora_key(k):
            continue
        tens = []
        base = None
        for sd in sds:
            if k in sd and isinstance(sd[k], torch.Tensor):
                tens.append(sd[k])
                base = sd[k] if base is None else base
            else:
                tens.append(torch.zeros_like(base) if base is not None else None)
        if any(t is None for t in tens):
            continue
        merged[k] = w_primary * tens[0] + w_partner * tens[1]

    # copy non-LoRA keys from primary (modules_to_save, etc.)
    for k, v in prim["sd"].items():
        if not _is_lora_key(k):
            merged[k] = v

    return merged

def load_adapter_from_repo(adapter_repo: str):
    """
    Load the base model + a LoRA adapter from Hugging Face / local repo,
    without using Unsloth. Uses the same BASE_MODEL_NAME and DTYPE.
    """
    # Load base model
    m = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=DTYPE,
    )

    # Attach LoRA adapter
    m = PeftModel.from_pretrained(m, adapter_repo)

    # Optional: disable cache for safety with some PEFT configs
    try:
        m.config.use_cache = False
    except Exception:
        pass

    # Make sure the active adapter is set
    try:
        m.set_adapter(m.active_adapter)
    except Exception:
        if hasattr(m, "peft_config") and len(m.peft_config):
            m.set_adapter(list(m.peft_config.keys())[0])

    return m.to(DEVICE).eval()

In [ ]:
@torch.no_grad()
def score_one(model, text: str, max_len: int = 256):
    enc = base_tokenizer([text], padding=True, truncation=True, max_length=max_len, return_tensors="pt")
    enc = {k: v.to(DEVICE) for k, v in enc.items()}
    out = model(input_ids=enc["input_ids"], attention_mask=enc["attention_mask"], labels=enc["input_ids"])
    loss = float(out.loss.item())
    ppl  = float(math.exp(min(20.0, loss)))
    toks = int(enc["attention_mask"].sum().item())
    return {"loss": loss, "ppl": ppl, "avg_logprob": -loss, "entropy": loss, "tokens": toks}

@torch.no_grad()
def score_one_base(text: str, max_len: int = 256):
    """
    Baseline: LLaMA only (no adapters).
    Uses emb_model directly as the base LM.
    """
    enc = base_tokenizer([text], padding=True, truncation=True,
                         max_length=max_len, return_tensors="pt")
    enc = {k: v.to(DEVICE) for k, v in enc.items()}
    out = emb_model(input_ids=enc["input_ids"],
                    attention_mask=enc["attention_mask"],
                    labels=enc["input_ids"])
    loss = float(out.loss.item())
    ppl  = float(math.exp(min(20.0, loss)))
    toks = int(enc["attention_mask"].sum().item())
    return {"loss": loss, "ppl": ppl, "avg_logprob": -loss, "entropy": loss, "tokens": toks}

def score_merge_two(text: str,
                    primary_lang: str,
                    partner_lang: str,
                    w_primary: float,
                    w_partner: float):
    """
    TWO-WAY merge Urdu + one partner (Arabic in this setup).
    Merge in CPU, load a temp model, score, then free.
    """
    s = w_primary + w_partner
    if s <= 0:
        # degenerate -> just use primary_lang adapter
        mdl = load_adapter_from_repo(ADAPTERS[primary_lang])
        metrics = score_one(mdl, text)
        del mdl; torch.cuda.empty_cache()
        return metrics

    w_primary = w_primary / s
    w_partner = w_partner / s

    merged_sd = soft_merge_two_sd(
        ADAPTERS[primary_lang], ADAPTERS[partner_lang],
        w_primary, w_partner
    )

    mdl = load_adapter_from_repo(ADAPTERS[primary_lang])
    active = getattr(mdl, "active_adapter", None) or list(mdl.peft_config.keys())[0]
    set_peft_model_state_dict(mdl, merged_sd, adapter_name=active, ignore_mismatched_sizes=True)

    metrics = score_one(mdl, text)
    del mdl; torch.cuda.empty_cache()
    return metrics

In [ ]:
single_ur_model = load_adapter_from_repo(ADAPTERS["ur"])
single_ar_model = load_adapter_from_repo(ADAPTERS["ar"])
print("[Single] Loaded Urdu and Arabic adapters once for all samples.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/project/6101777/wangzn/adapter_mllm/.venv/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)


[Single] Loaded Urdu adapter once for all samples.


In [ ]:
# ============================================================
# Per-input adaptive evaluation
# ============================================================
RESULTS = []
t_start = time.time()

# Save results inside the project
RESULTS_DIR = PROJECT_ROOT / "code_mixing_results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Unique run ID: hostname + PID + timestamp
INTERMEDIATE_OUT_CSV = RESULTS_DIR / f"ur_eval_intermediate.csv"


last_flush_idx = 0  # index in RESULTS up to which we've already saved

for i, txt in enumerate(eval_texts):
    preview = txt[:160].replace("\n", " ")
    evec = embed_one(txt)
    per_lang = compute_per_lang_sims(txt, evec)

    # --- BASELINE: LLaMA only (no adapters) ---
    base_metrics = score_one_base(txt)
    RESULTS.append({
        "idx": i, "text_preview": preview,
        "target_lang": TARGET_LANG,
        "system": "base",
        "sim_mode": "none",
        "alpha_bpe": None, "beta_emb": None,
        "choice_1_lang": None, "choice_1_weight": None,
        "choice_2_lang": None, "choice_2_weight": None,
        "choice_3_lang": None, "choice_3_weight": None,
        "top_score": None, "safety_fallback": False,
        **base_metrics,
    })

    # --- SINGLE baseline (Urdu adapter) ---
    single_ur_metrics = score_one(single_ur_model, txt)
    RESULTS.append({
        "idx": i, "text_preview": preview,
        "target_lang": TARGET_LANG,
        "system": "single_ur",
        "sim_mode": "none",
        "alpha_bpe": None, "beta_emb": None,
        "choice_1_lang": "ur", "choice_1_weight": 1.0,
        "choice_2_lang": None, "choice_2_weight": None,
        "choice_3_lang": None, "choice_3_weight": None,
        "top_score": None, "safety_fallback": False,
        **single_ur_metrics,
    })

    # --- SINGLE baseline (Arabic adapter) ---
    single_ar_metrics = score_one(single_ar_model, txt)
    RESULTS.append({
        "idx": i, "text_preview": preview,
        "target_lang": TARGET_LANG,
        "system": "single_ar",
        "sim_mode": "none",
        "alpha_bpe": None, "beta_emb": None,
        "choice_1_lang": "ar", "choice_1_weight": 1.0,
        "choice_2_lang": None, "choice_2_weight": None,
        "choice_3_lang": None, "choice_3_weight": None,
        "top_score": None, "safety_fallback": False,
        **single_ar_metrics,
    })

    # --- FIXED MERGES: 50/50, 75/25, 25/75 Urdu–Arabic (on the fly) ---
    fixed_configs = [
        ("fixed_ur50_ar50", 0.50, 0.50),
        ("fixed_ur75_ar25", 0.75, 0.25),
        ("fixed_ur25_ar75", 0.25, 0.75),
    ]

    for system_name, w_ur, w_ar in fixed_configs:
        fixed_metrics = score_merge_two(
            txt,
            primary_lang="ur",
            partner_lang="ar",
            w_primary=w_ur,
            w_partner=w_ar,
        )

        RESULTS.append({
            "idx": i, "text_preview": preview,
            "target_lang": TARGET_LANG,
            "system": system_name,
            "sim_mode": "fixed",
            "alpha_bpe": None, "beta_emb": None,
            "choice_1_lang": "ur", "choice_1_weight": w_ur,
            "choice_2_lang": "ar", "choice_2_weight": w_ar,
            "choice_3_lang": None, "choice_3_weight": None,
            "top_score": None,
            "safety_fallback": False,
            **fixed_metrics,
        })

    # --- ADAPTIVE per scheme (Urdu + Arabic; weights from similarity) ---
    for scheme, (alpha, beta) in SIM_SCHEMES.items():
        chosen, weights, top_score, safety = select_adapters(per_lang, scheme, TARGET_LANG)
        langs = [c["lang"] for c in chosen]

        # Map weights to Urdu + Arabic
        w_ur = 0.0
        w_ar = 0.0
        for lg, w in zip(langs, weights):
            if lg == "ur":
                w_ur = w
            elif lg == "ar":
                w_ar = w

        partners = [lg for lg in langs if lg != TARGET_LANG]

        if len(partners) == 0 or w_ar <= 0.0:
            # No useful Arabic partner → fall back to single Urdu
            metrics = single_ur_metrics
        else:
            metrics = score_merge_two(
                txt,
                primary_lang="ur",
                partner_lang="ar",
                w_primary=w_ur,
                w_partner=w_ar,
            )

        RESULTS.append({
            "idx": i, "text_preview": preview,
            "target_lang": TARGET_LANG,
            "system": f"adaptive:{scheme}",
            "sim_mode": scheme,
            "alpha_bpe": alpha, "beta_emb": beta,
            "choice_1_lang": langs[0] if len(langs) > 0 else None,
            "choice_1_weight": weights[0] if len(weights) > 0 else None,
            "choice_2_lang": langs[1] if len(langs) > 1 else None,
            "choice_2_weight": weights[1] if len(weights) > 1 else None,
            "choice_3_lang": None,
            "choice_3_weight": None,
            "top_score": float(top_score),
            "safety_fallback": bool(safety),
            **metrics,
        })

    # ---- PERIODIC FLUSH ----
    if (i+1) % 5 == 0:
        elapsed = (time.time() - t_start)/60
        print(f"[{i+1}/{len(eval_texts)}] done | elapsed ~{elapsed:.1f} min")

        new_rows = RESULTS[last_flush_idx:]
        if new_rows:
            intermediate_df = pd.DataFrame(new_rows)
            header = not os.path.exists(INTERMEDIATE_OUT_CSV)
            intermediate_df.to_csv(INTERMEDIATE_OUT_CSV, mode='a', header=header, index=False)
            last_flush_idx = len(RESULTS)
            print(f"  Intermediate results saved to {INTERMEDIATE_OUT_CSV}")

# Final flush for any leftover rows
if last_flush_idx < len(RESULTS):
    leftover_df = pd.DataFrame(RESULTS[last_flush_idx:])
    header = not os.path.exists(INTERMEDIATE_OUT_CSV)
    leftover_df.to_csv(INTERMEDIATE_OUT_CSV, mode='a', header=header, index=False)
    print(f"Final intermediate flush: saved remaining {len(RESULTS) - last_flush_idx} rows.")

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Final intermediate flush: saved remaining 24 rows.


In [ ]:
# ============================================================
# Save + summary (per-run file, safe for parallel jobs)
# ============================================================
from pathlib import Path
from datetime import datetime
import socket
import os

df = pd.DataFrame(RESULTS)

# Save results inside the project
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Unique run ID: hostname + PID + timestamp
run_id = f"{socket.gethostname()}_{os.getpid()}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
out_csv = RESULTS_DIR / f"ur_eval_{run_id}.csv"

df.to_csv(out_csv, index=False)
print("\n✅ Saved per-input results:", out_csv)

summary = (
    df.groupby("system")["ppl"]
      .mean()
      .reset_index()
      .sort_values("ppl")
)

print("\n=== Avg PPL per system ===")
display(summary)

df.head()